In [1]:
from lusidtools.jupyter_tools import toggle_code

"""Cash ladder

Demonstration of how to compute a cash ladder for a portfolio.

Attributes
----------
bucketed cashflow
cash
cash ladder
"""

toggle_code("Toggle Docstring")

# Cash Ladder

In this notebook, we demonstrate how users can create a cash ladder with LUSID. For the purposes of this notebook, we will first upsert a portfolio and some example transactions. Then we will use bucketed cashflows to create a cash ladder. This will help us understand how those transactions have affected our balance over time.

For the purposes of demonstation it is assumed that the current timestamp is 02/04/2020 00:00:00.  This means that there is no data uploaded to LUSID with an effective date after this point (it would be in the "future").

## Setup LUSID

We start by importing relevant libraries, authenticating our user, and creating our API client.

In [35]:
# Import LUSID packages
from lusidtools.pandas_utils.lusid_pandas import lusid_response_to_data_frame
from lusidtools.cocoon.seed_sample_data import seed_data
from lusidjam.refreshing_token import RefreshingToken
from lusid.utilities import ApiClientFactory
import lusid.models as models
import lusid.api as la
import lusid
from IPython.core.display import HTML
from IPython.display import display

# Import general purpose packages
import os
import json
import pandas as pd
import numpy as np
import pytz
import warnings
from datetime import datetime, timedelta

# Configure notebook warnings and pandas display
warnings.filterwarnings("ignore")
pd.set_option("display.max_columns", None)

# Authenticate our user and create our API client
secrets_path = os.getenv("FBN_SECRETS_PATH")

api_factory = lusid.utilities.ApiClientFactory(
    token=RefreshingToken(),
    api_secrets_filename=secrets_path,
    app_name="LusidJupyterNotebook",
)
api_url = api_factory.api_client.configuration._base_path.replace("api", "")
print("LUSID Environment :", api_url)

LUSID Environment : https://fbn-carlfazekas.lusid.com/


In [19]:
# Define the relevant APIs we will use
transaction_portfolio_api = api_factory.build(la.TransactionPortfoliosApi)
quotes_api = api_factory.build(la.QuotesApi)
configuration_recipe_api = api_factory.build(la.ConfigurationRecipeApi)
portfolios_api = api_factory.build(la.PortfoliosApi)
property_definition_api = api_factory.build(la.PropertyDefinitionsApi)

## 1. Prepare & Setup Data 

In this notebook, we have a portfolio called EQUITY_FUND. We will create the portfolio with two sub-holding keys, which we will then use later in the notebook to optionally group our cashflows and cash ladder.

In [67]:
# Portfolio scope and code
scope = "cash-ladder-001"
portfolio_code = "EQUITY_FUND"

In [68]:
# Create a list containing two property definition objects
property_definitions = []

property_definitions.append(
    models.CreatePropertyDefinitionRequest(
        domain="Transaction",
        scope=scope,
        code="Strategy",
        value_required=False,
        display_name="Strategy",
        data_type_id=models.ResourceId(scope="system", code="string"),
        property_description="The strategy associated with the transaction.",
    )
)

property_definitions.append(
    models.CreatePropertyDefinitionRequest(
        domain="Transaction",
        scope=scope,
        code="CashAccount",
        value_required=False,
        display_name="Cash Account",
        data_type_id=models.ResourceId(scope="system", code="string"),
        property_description="The external cash account associated with the transaction.",
    )
)

# Upsert the property definitions, store the key in another list
sub_holding_key_properties = []

for property_definition in property_definitions:
    # Call LUSID API and log response detail if exception occurs
    try:
        api_response = property_definition_api.create_property_definition(
            create_property_definition_request=property_definition
        )
        # Confirm success
        display(
            f"{api_response.display_name} property definition created successfully."
        )
    except lusid.ApiException as e:
        display(json.loads(e.body)["title"])

    # Store the property keys for future use
    sub_holding_key_properties.append(
        f"{property_definition.domain}/{property_definition.scope}/{property_definition.code}"
    )

'Strategy property definition created successfully.'

'Cash Account property definition created successfully.'

In [22]:
# Load a mapping file for loading data
with open("config/cash_ladder_seed_data.json", "r") as seed_data_config:
    mapping_file = json.loads(seed_data_config.read())

# Load a file of equity transactions
transactions_file = r"data/cash_ladder.csv"
transactions_df = pd.read_csv(transactions_file)
transactions_df["portfolio_code"] = portfolio_code

# Load portfolios, instruments, and transactions
seed_data_response = seed_data(
    api_factory=api_factory,
    domains=["portfolios", "instruments", "transactions"],
    scope=scope,
    transaction_file=transactions_file,
    file_type="csv",
    mappings=mapping_file,
    sub_holding_keys=sub_holding_key_properties,
)

## 2. Upload Quote Data

We will load in some quotes for our equity instruments.

In [23]:
equity_prices = pd.read_csv("data/equity_quotes.csv")
equity_prices.head(n=5)

,instrument_id,instrument_name,date,price,currency
0,EQ_GB001,vodafone,29/03/2020,114.60,GBP
1,EQ_GB001,vodafone,30/03/2020,114.50,GBP
2,EQ_GB001,vodafone,31/03/2020,118.54,GBP
3,EQ_GB001,vodafone,01/04/2020,117.64,GBP
4,EQ_GB002,Anglo American plc,29/03/2020,1816.40,GBP


We will then upsert these quotes to LUSID.

In [66]:
# Iterate through our dataframe and add each quote to an UpsertQuoteRequest
instrument_quotes = {
    index: models.UpsertQuoteRequest(
        quote_id=models.QuoteId(
            quote_series_id=models.QuoteSeriesId(
                provider="Lusid",
                instrument_id=row["instrument_id"],
                instrument_id_type="ClientInternal",
                quote_type="Price",
                field="mid",
            ),
            effective_at=str(
                pytz.timezone("UTC")
                .localize(dt=datetime.strptime(row["date"], "%d/%m/%Y"))
                .isoformat()
            ),
        ),
        metric_value=models.MetricValue(value=row["price"], unit=row["currency"]),
        scale_factor=1,
    )
    for index, row in equity_prices.iterrows()
}

# Upsert the quotes
response = quotes_api.upsert_quotes(scope=scope, request_body=instrument_quotes)

if response.failed == {}:
    display(
        f"Quotes successfully loaded into LUSID. {len(response.values)} quotes loaded."
    )
else:
    display(
        f"Some failures occurred during quote upsertion, {len(response.failed)} did not get loaded into LUSID."
    )

'Quotes successfully loaded into LUSID. 12 quotes loaded.'

## 3. Upsert Configuration Recipe

Create and upsert basic configuration recipe to be used for valuation.

In [113]:
# Create a configuration recipe
configuration_recipe = models.ConfigurationRecipe(
    scope=scope,
    code=portfolio_code,
    # Define the market rules and which quotes to use
    market=models.MarketContext(
        market_rules=[
            models.MarketDataKeyRule(
                key="Quote.ClientInternal.*",
                supplier="Lusid",
                data_scope=scope,
                quote_type="Price",
                field="mid",
                quote_interval="2D.0D",
            )
        ],
        options=models.MarketOptions(
            default_supplier="Lusid",
            default_instrument_code_type="ClientInternal",
            default_scope=scope,
        ),
    ),
)

# Upsert the recipe to LUSID
config_recipe_response = configuration_recipe_api.upsert_configuration_recipe(
    upsert_recipe_request=models.UpsertRecipeRequest(
        configuration_recipe=configuration_recipe
    )
)

## 4. Produce Cash Ladder

### 4.1 Get Bucketed Cash Flows

We will use the `GetBucketedCashFlows` API endpoint to acquire the data for our cash ladder. This aggregates a transaction portfolio's instruments by date. See API documentation for reference: https://www.lusid.com/docs/api/#operation/GetBucketedCashFlows

In [114]:
# Set the start and end dates for our cash ladder
start_date = datetime(year=2020, month=4, day=2, tzinfo=pytz.UTC)
end_date = datetime(year=2020, month=4, day=10, tzinfo=pytz.UTC)

# Create a string with the current time
time_now = datetime.utcnow().strftime("%H:%M:%S")

# Create a request to get the bucketed cashflows
bucket_response = transaction_portfolio_api.get_bucketed_cash_flows(
    scope=scope,
    code=portfolio_code,
    bucketed_cash_flow_request=models.BucketedCashFlowRequest(
        # When bucketing, there is not a unique way to allocate the bucket points. Supported options are: [RoundDown, RoundUp]
        rounding_method="RoundDown",
        # List every day between our start and end dates
        bucketing_dates=[
            start_date + timedelta(days=x) for x in range((end_date - start_date).days)
        ],
        # Set the valuation recipe
        recipe_id=models.ResourceId(
            scope=scope,
            code=portfolio_code,
        ),
        # Group by date and currency
        group_by=["Valuation/CashFlowDate/RoundDown",
                  "Valuation/CashFlowCurrency"],
        # Add several addresses for more information
        addresses=[
            "Valuation/CashFlowAmount",
            "Valuation/OpenBalance",
            "Valuation/CloseBalance",
            "Valuation/ActivityType",
            "Valuation/Movements",
        ],
        # Set effective date, cash flow type, and start window
        effective_at=start_date,
        cash_flow_type="PortfolioCashFlow",
    ),
)

### 4.2 Build Cash Ladder

Now we have our bucketed cashflows, we will extract the cash impacts and create our cash ladder.

#### Data Preprocessing

First we need to extract the relevant information from the response.

In [115]:
# Mapping function used to extract value and unit pairs from a 0D column
def format_result0D_column(values: pd.Series):
    return [(x["value"], x["units"]) for x in values]


# Convert the response to a pandas dataframe
data = pd.DataFrame.from_records(np.array(bucket_response.data))

# Use the mapping function to format CashFlowAmount
data["Append(Valuation/CashFlowAmount)"] = data["Append(Valuation/CashFlowAmount)"].map(
    format_result0D_column
)

# Group rows by currency
data = data.groupby(["Valuation/CashFlowCurrency"], sort=False)


# Define function to convert a dataframe row to a record
def to_record(row):
    # We make use of the CashLadderRecord model
    return models.CashLadderRecord(
        effective_date=pd.to_datetime(row["Valuation/CashFlowDate/RoundDown"]),
        open=row["Valuation/OpenBalance"],
        close=row["Valuation/CloseBalance"],
        activities=list(
            zip(
                row["Append(Valuation/Movements)"],
                row["Append(Valuation/CashFlowAmount)"],
            )
        ),
    )

#### Constructing Cash Ladders

Now we can construct and view the cash ladder for each currency with available quote data. In this case, we have two available: GBP and USD.

In [116]:
def construct_cash_ladder(data):
    # Iterate through each dataframe group (i.e. currency)
    for key, df in data:
        currency = key[0]
        print(f"Cash ladder for currency {currency}:")

        # Create a new dataframe for each cash ladder
        ladder = pd.DataFrame(
            columns=[
                "ReportRunTime",
                "SettlementDate",
                "Currency",
                "Portfolio",
                "CashLadderStatus",
                "CashImpact",
            ]
        )

        # Iterate through each cash row in our dataframe
        for _, row in df.iterrows():
            if row["Valuation/ActivityType"] != "Activity":
                continue
            # Convert the row to a CashLadderRecord
            record = to_record(row)

            # Create a new row for our SOD balance
            ladder.loc[len(ladder)] = {
                "ReportRunTime": time_now,
                "SettlementDate": record.effective_date.strftime("%Y-%m-%d"),
                "Currency": currency,
                "Portfolio": portfolio_code,
                "CashLadderStatus": "Open Balance",
                "CashImpact": record.open,
            }

            # Iterate through each activity
            for item in record.activities:
                if item[0] != "Closed" and item[0] != "Open":
                    # Create a new row for each cash impact
                    ladder.loc[len(ladder)] = {
                        "ReportRunTime": time_now,
                        "SettlementDate": record.effective_date.strftime("%Y-%m-%d"),
                        "Currency": item[1][1][1:4],
                        "Portfolio": portfolio_code,
                        "CashLadderStatus": f"Cash impact from {item[0]}",
                        "CashImpact": item[1][0],
                    }

            # Create a new row for our EOD balance
            ladder.loc[len(ladder)] = {
                "ReportRunTime": time_now,
                "SettlementDate": record.effective_date.strftime("%Y-%m-%d"),
                "Currency": currency,
                "Portfolio": portfolio_code,
                "CashLadderStatus": "Closed Balance",
                "CashImpact": record.close,
            }

        # Display the cash ladder for this currency
        display(ladder.set_index(["SettlementDate", "Portfolio"]))


construct_cash_ladder(data)

Cash ladder for currency USD:


ReportRunTime Currency        CashLadderStatus  \
SettlementDate Portfolio                                                    
2020-04-03     EQUITY_FUND      10:59:47      USD            Open Balance   
               EQUITY_FUND      10:59:47      USD  Cash impact from Side2   
               EQUITY_FUND      10:59:47      USD  Cash impact from Side1   
               EQUITY_FUND      10:59:47      USD          Closed Balance   
2020-04-04     EQUITY_FUND      10:59:47      USD            Open Balance   
               EQUITY_FUND      10:59:47      USD  Cash impact from Side2   
               EQUITY_FUND      10:59:47      USD          Closed Balance   

                            CashImpact  
SettlementDate Portfolio                
2020-04-03     EQUITY_FUND  17000000.0  
               EQUITY_FUND   1800000.0  
               EQUITY_FUND  -2000000.0  
               EQUITY_FUND  16800000.0  
2020-04-04     EQUITY_FUND  16800000.0  
               EQUITY_FUND  -3000000.0  
               EQUITY_FUND  13800000.0

Cash ladder for currency GBP:


ReportRunTime Currency        CashLadderStatus  \
SettlementDate Portfolio                                                    
2020-04-03     EQUITY_FUND      10:59:47      GBP            Open Balance   
               EQUITY_FUND      10:59:47      GBP  Cash impact from Side2   
               EQUITY_FUND      10:59:47      GBP  Cash impact from Side1   
               EQUITY_FUND      10:59:47      GBP          Closed Balance   
2020-04-04     EQUITY_FUND      10:59:47      GBP            Open Balance   
               EQUITY_FUND      10:59:47      GBP  Cash impact from Side1   
               EQUITY_FUND      10:59:47      GBP          Closed Balance   
2020-04-05     EQUITY_FUND      10:59:47      GBP            Open Balance   
               EQUITY_FUND      10:59:47      GBP  Cash impact from Side1   
               EQUITY_FUND      10:59:47      GBP          Closed Balance   
2020-04-06     EQUITY_FUND      10:59:47      GBP            Open Balance   
               EQUITY_FUND      10:59:47      GBP  Cash impact from Side1   
               EQUITY_FUND      10:59:47      GBP          Closed Balance   

                            CashImpact  
SettlementDate Portfolio                
2020-04-03     EQUITY_FUND  10300000.0  
               EQUITY_FUND  -6900000.0  
               EQUITY_FUND  -1000000.0  
               EQUITY_FUND   2400000.0  
2020-04-04     EQUITY_FUND   2400000.0  
               EQUITY_FUND   1000000.0  
               EQUITY_FUND   3400000.0  
2020-04-05     EQUITY_FUND   3400000.0  
               EQUITY_FUND   3000000.0  
               EQUITY_FUND   6400000.0  
2020-04-06     EQUITY_FUND   6400000.0  
               EQUITY_FUND  20000000.0  
               EQUITY_FUND  26400000.0

### 4.3 Get Filtered Bucketed Cash Flows

Now we will build another cash ladder, but this time only including transactions using the EquityGrowth Strategy.

First we'll again request Bucketed Cash Flows but with an additional filter provided for the desired sub-holding key.

In [117]:
# Set the start and end dates for our cash ladder
start_date = datetime(year=2020, month=4, day=2, tzinfo=pytz.UTC)
end_date = datetime(year=2020, month=4, day=10, tzinfo=pytz.UTC)

# Create a string with the current time
time_now = datetime.utcnow().strftime("%H:%M:%S")

# Create a request to get the bucketed cashflows
bucket_response = transaction_portfolio_api.get_bucketed_cash_flows(
    scope=scope,
    code=portfolio_code,
    bucketed_cash_flow_request=models.BucketedCashFlowRequest(
        # When bucketing, there is not a unique way to allocate the bucket points. Supported options are: [RoundDown, RoundUp]
        rounding_method="RoundDown",
        # List every day between our start and end dates
        bucketing_dates=[
            start_date + timedelta(days=x) for x in range((end_date - start_date).days)
        ],
        # Set the valuation recipe
        recipe_id=models.ResourceId(
            scope=scope,
            code=portfolio_code,
        ),
        # Group by date and currency
        group_by=["Valuation/CashFlowDate/RoundDown",
                  "Valuation/CashFlowCurrency"],
        # Add several addresses for more information
        addresses=[
            "Valuation/CashFlowAmount",
            "Valuation/OpenBalance",
            "Valuation/CloseBalance",
            "Valuation/ActivityType",
            "Valuation/Movements",
        ],
        # Set effective date, cash flow type, and start window
        effective_at=start_date,
        cash_flow_type="PortfolioCashFlow",
        filter=f"subHoldingKeys.Transaction/{scope}/Strategy eq 'EquityGrowth'",
    ),
)

# Convert the response to a pandas dataframe
data = pd.DataFrame.from_records(np.array(bucket_response.data))

# Use the mapping function to format CashFlowAmount
data["Append(Valuation/CashFlowAmount)"] = data["Append(Valuation/CashFlowAmount)"].map(
    format_result0D_column
)

# Group rows by currency
data = data.groupby(["Valuation/CashFlowCurrency"], sort=False)

With a new response in our dataframe, we can call the previously defined functions again to build and display our cash ladder.

In [118]:
construct_cash_ladder(data)

Cash ladder for currency GBP:


ReportRunTime Currency        CashLadderStatus  \
SettlementDate Portfolio                                                    
2020-04-03     EQUITY_FUND      10:59:48      GBP            Open Balance   
               EQUITY_FUND      10:59:48      GBP  Cash impact from Side2   
               EQUITY_FUND      10:59:48      GBP  Cash impact from Side1   
               EQUITY_FUND      10:59:48      GBP          Closed Balance   
2020-04-04     EQUITY_FUND      10:59:48      GBP            Open Balance   
               EQUITY_FUND      10:59:48      GBP  Cash impact from Side1   
               EQUITY_FUND      10:59:48      GBP          Closed Balance   
2020-04-05     EQUITY_FUND      10:59:48      GBP            Open Balance   
               EQUITY_FUND      10:59:48      GBP  Cash impact from Side1   
               EQUITY_FUND      10:59:48      GBP          Closed Balance   
2020-04-06     EQUITY_FUND      10:59:48      GBP            Open Balance   
               EQUITY_FUND      10:59:48      GBP  Cash impact from Side1   
               EQUITY_FUND      10:59:48      GBP          Closed Balance   

                            CashImpact  
SettlementDate Portfolio                
2020-04-03     EQUITY_FUND    300000.0  
               EQUITY_FUND  -6000000.0  
               EQUITY_FUND  -3000000.0  
               EQUITY_FUND  -8700000.0  
2020-04-04     EQUITY_FUND  -8700000.0  
               EQUITY_FUND   1000000.0  
               EQUITY_FUND  -7700000.0  
2020-04-05     EQUITY_FUND  -7700000.0  
               EQUITY_FUND   3000000.0  
               EQUITY_FUND  -4700000.0  
2020-04-06     EQUITY_FUND  -4700000.0  
               EQUITY_FUND  20000000.0  
               EQUITY_FUND  15300000.0

Cash ladder for currency USD:


ReportRunTime Currency        CashLadderStatus  \
SettlementDate Portfolio                                                    
2020-04-03     EQUITY_FUND      10:59:48      USD            Open Balance   
               EQUITY_FUND      10:59:48      USD  Cash impact from Side2   
               EQUITY_FUND      10:59:48      USD          Closed Balance   

                            CashImpact  
SettlementDate Portfolio                
2020-04-03     EQUITY_FUND  -3000000.0  
               EQUITY_FUND   3000000.0  
               EQUITY_FUND         0.0

In [119]:
display(HTML("<h2>5. Links</h2>"))

display(
    HTML(
        f'<a href="{api_url}app/dashboard/transactions?scope=cash-ladder-001&code=EQUITY_FUND&entityType=Portfolio&effectiveDate=2023-10-17T16:03:24.218Z&dateFrom=2000-01-01T00:00:00.000Z&dateTo=2023-10-17T16:03:24.218Z&taxLots=false&withOrders=false&withAllocations=false&dashboardType=input&showNonActiveTransactions=false" target="_blank">Transactions</a>'
    )
)

display(
    HTML(
        f'<a href="{api_url}app/dashboard/cash-ladder?scope=cash-ladder-001&code=EQUITY_FUND&entityType=Portfolio&recipeScope=cash-ladder-001&recipeCode=EQUITY_FUND&effectiveDate=2020-04-01T23:01:00.000Z&fromDate=2020-04-01T23:00:00.000Z&toDate=2023-10-31T23:59:59.999Z&allDates=false" target="_blank">Cash Ladder</a>'
    )
)